This notebook exists to investigate the need of the parameters included in the dataset. Each parameter is tested against the others to see if they are statistically correlated....
Then wrapper methods are used to determine which ones are most important....
Want to reduce number of parameters needed to produce a good output so to reduce the cahnce of overfitting and increase the generalisation of the model.

In [20]:
%run "Parameter_Estimation.ipynb" #allowing access to parameters

100%|█████████████████████████████████████████████████████████████████████████████████| 88/88 [00:00<00:00, 162.15it/s]


In [21]:
import itertools
from scipy.stats import pearsonr

In [22]:
def investigate_corr(params):
    """
    uses pearson correlation test to determine wether too parameters are correlated at the 5% significance level
    
    """
    for key1, key2 in itertools.combinations(params.keys(), 2):
        corr, p_value = pearsonr(params[key1], params[key2])
        if p_value < 0.05:
            print(f"parameter {key1} and parameter {key2} are significantly correlated, p = {p_value}, corr = {corr}")
    return None

#investigate_corr(params)

The above suggests that rr_std is not neccessary to be included, aswell as either pNN50 and RMSSD

In [19]:
removed_params = ['rr_std', 'RMSSD', 'pNN50', 'mean', 'lf', 'hf', 'skews', 'rr_mean', 'std', 'power_ratio', 'shannon_en'] #convert to a set?

selected_params = {}
for key in params.keys():
    if key not in removed_params:
        selected_params[key] = params[key]
        